# CDS 303 Group 3 Project: Phishing and Spam Categorization Code
### Outline of Code
- Importing the Cleaned Data With Unique Features
- Logistic Regression
- Exploration of Alternative Models
- Importing Tokenized and Vectorized Emails
- Apply the same models

## Importing the Cleaned Data

In [29]:
# Importing Libraries
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn import datasets, model_selection, metrics
from sklearn import linear_model, naive_bayes, tree
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Importing Data
df = pd.read_csv("../CDS303/CSV Files/Post_EDA_encoded_df.csv")
df.head()

from sklearn.preprocessing import StandardScaler

# Scale the data
y = df['Safe_Email']
X = df.drop(['Safe_Email'], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Splitting the Data, Training versus Testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled, y, test_size=0.3)

X_train_pct = round((len(X_train) / (len(X_train) + len(X_test))), 2)
X_test_pct = round((len(X_test) / (len(X_train) + len(X_test))), 2)

print('Classification training dataset shape is:', X_train.shape, 'Testing dataset shape is:', X_test.shape)
print('Regression train/test split is:', X_train_pct, '/', X_test_pct)

Classification training dataset shape is: (12670, 19) Testing dataset shape is: (5431, 19)
Regression train/test split is: 0.7 / 0.3


## Logistic Regression
### Creation and Tuning

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
# Define your logistic regression model
logistic_model = linear_model.LogisticRegression()

# Stratified K-Fold cross-validation
from sklearn.model_selection import StratifiedKFold
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds)

# Initialize lists to store evaluation metrics for each fold
fold_results = []

for fold_idx, (train_index, val_index) in enumerate(skf.split(X_train, y_train), 1):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    
    # Train the model on the training fold
    logistic_model.fit(X_train_fold, y_train_fold)
    
    # Make predictions on the validation fold
    y_pred_fold = logistic_model.predict(X_val_fold)
    
    # Evaluate the model on the validation fold
    accuracy = accuracy_score(y_val_fold, y_pred_fold)
    precision = precision_score(y_val_fold, y_pred_fold)
    recall = recall_score(y_val_fold, y_pred_fold)
    f1 = f1_score(y_val_fold, y_pred_fold)
    
    # Store the results for this fold
    fold_results.append({'Fold': fold_idx,
                         'Accuracy': accuracy,
                         'Precision': precision,
                         'Recall': recall,
                         'F1 Score': f1})

# Create a DataFrame to store the fold-wise results
results_df = pd.DataFrame(fold_results)

# Calculate mean scores across all folds
mean_scores = results_df.mean(axis=0)

# Create a DataFrame for mean scores
mean_scores_df = pd.DataFrame(mean_scores).transpose()

# Set the 'Fold' column to 'Mean' for the mean scores DataFrame
mean_scores_df['Fold'] = 'Mean'

# Concatenate the mean scores DataFrame with the original results DataFrame
results_df = pd.concat([results_df, mean_scores_df], ignore_index=True)

# Display the results DataFrame
results_df

,Fold,Accuracy,Precision,Recall,F1 Score
0,1,0.746646,0.724053,0.947843,0.820970
1,2,0.756117,0.736469,0.937540,0.824929
2,3,0.752170,0.734415,0.933033,0.821894
3,4,0.756511,0.733766,0.945911,0.826442
4,5,0.769534,0.739259,0.963941,0.836780
5,Mean,0.756196,0.733593,0.945654,0.826203


We fit the model, but we have unbalanced classes. Because there are approximately 11,000 safe emails versus 7,000 unsafe emails, we use stratified k-fold cross validation.

The model has a mean accuracy of .75 across all 5 of the folds. This is the overall correctness of the model. It is measured by taking the overall correct predictions divided by the total number of cases examined. Precision refers to the accuracy of the positive predictions. Recall is important when the cost of a false negative is high because it is a measure of the actual positives identified by the model. Finally, the F-Score capture the precision and recall in one number.

- Accuracy= TP+TN+FP+FN / TP+TN

- Precision= TP / TP+FP

- Recall= TP / TP+FN

- F1 Score= 2 x (Precision x Recall) / (Precision + Recall)

This Accuracy score seems quite low for the use case. We would like to predict the type of emails at least 95% of the time. The following section we explore the hyperparameter space for Logarithmic Regression.

#### Hyperparameter Search
There are 5 different hyperparameters for the logistic regression model. We perform a grid search to find the best hyperparameter. We will use the F1 score as our primary consideration for comparing the models with different hyperparameters because we have imbalanced classes and want to account for both the perfomance of the model on precision and recall.

One significant Hyperparameter is the C (Inverse Regularization Strength). This is used to prevent overfitting. A high value of C means there is less regularization and the risk of overfitting. A low vaule means there is a risk of underfitting.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

import warnings
warnings.filterwarnings("ignore")
from sklearn.exceptions import FitFailedWarning

# Ignore FitFailedWarning, because some of the combinations of hyperparameters are expected to not converge
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FitFailedWarning)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Logistic Regression model
model = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence

# Grid of parameters to search over
param_grid = [
    {'penalty': ['l1', 'l2'], 'solver': ['liblinear'], 'C': [0.001, 0.01, 0.1, 1, 10, 100]},
    {'penalty': ['l2', 'none'], 'solver': ['newton-cg', 'lbfgs', 'sag'], 'C': [0.001, 0.01, 0.1, 1, 10, 100]},
    {'penalty': ['elasticnet'], 'solver': ['saga'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': np.linspace(0, 1, 10)}
]

# GridSearchCV to find the best parameters, score on f1 to balance the consideration of performance on recall and precision
clf = GridSearchCV(model, param_grid, cv=5, scoring='f1')
clf.fit(X_train, y_train)

# Output the best parameters and best score
print("Best parameters:", clf.best_params_)
print("Best cross-validation score: {:.2f}".format(clf.best_score_))


### Testing and Interpretation of Results

## Comparison to Other Models

## Logistic Regression on Email Text Data
### The Process

### Training

### Results